<a href="https://colab.research.google.com/github/mattekudacy/fcc-machine-learning-with-python/blob/main/Play_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os 
import numpy as np

In [ ]:
# path_to_file = tf.keras.utils.get_file('shakespeare.text', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving shrek.txt to shrek (1).txt


In [ ]:
# read and decode
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# count length of text which will be number of characeters
print("Length of text: {} character".format(len(text)))

Length of text: 74691 character


In [ ]:
print(text[:250])

SHREK (2001)
Writer Credits:
William Steig (Book); Ted Elliott, Terry Rossio, Joe Stillman & Roger S.H. Schulman (Screenplay); Cody Cameron, Chris Miller & Conrad Vernon (Additional Dialogue).
OPENING SEQUENCE: A FAIRYTALE BOOK
Voiceover: Shrek



In [ ]:
vocab = sorted(set(text))
# creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: SHREK (2001)
Encoded: [41 30 40 27 33  2  7 15 13 13 14  8  1]


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

SHREK (2001)


In [ ]:
seq_length = 100 #length of sequence for a training example
example_per_epoch = len(text)//(seq_length+1)

# create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk): # for example hello
  input_text = chunk[:-1] #hell
  target_text = chunk[1:] #ello
  return input_text, target_text #hell, ello

dataset = sequences.map(split_input_target) # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
SHREK (2001)
Writer Credits:
William Steig (Book); Ted Elliott, Terry Rossio, Joe Stillman & Roger

OUTPUT
HREK (2001)
Writer Credits:
William Steig (Book); Ted Elliott, Terry Rossio, Joe Stillman & Roger 


EXAMPLE

INPUT
S.H. Schulman (Screenplay); Cody Cameron, Chris Miller & Conrad Vernon (Additional Dialogue).
OPENI

OUTPUT
.H. Schulman (Screenplay); Cody Cameron, Chris Miller & Conrad Vernon (Additional Dialogue).
OPENIN


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) #vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (64, None, 256)           21248     
                                                                 
 lstm_7 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_7 (Dense)             (64, None, 83)            85075     
                                                                 
Total params: 5,353,299
Trainable params: 5,353,299
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) #ask model for a prediction on our first batch of training data
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") # print out input shape

(64, 100, 83) # (batch_size, sequence_length, vocab_size)


In [ ]:
# examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-0.0060461  -0.00045261  0.00296543 ... -0.0002754  -0.00076718
   0.00540906]
 [ 0.0022795   0.00388176  0.0044597  ... -0.0044797   0.00013975
   0.00113538]
 [ 0.00242784  0.00415331  0.00607896 ... -0.00365103 -0.00047672
   0.00094596]
 ...
 [ 0.00302848 -0.01179647 -0.00342689 ...  0.00590663 -0.00737848
   0.00703521]
 [-0.00075369 -0.01417141 -0.00557973 ...  0.0067171  -0.00958492
   0.00756675]
 [ 0.0019905  -0.01141972 -0.00306396 ...  0.01080399 -0.00201175
   0.01022684]], shape=(100, 83), dtype=float32)


In [ ]:
# look prediction at first time step
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

83
tf.Tensor(
[-6.0461015e-03 -4.5261148e-04  2.9654256e-03 -1.6953758e-03
  5.7952679e-03  2.1747346e-03 -7.6204771e-04  7.8829825e-03
  1.8282798e-03 -4.6868376e-03  6.9249352e-04  5.9049984e-05
 -2.4918835e-03 -2.4012490e-03  9.6842193e-04  3.7427070e-03
 -1.5273655e-04 -3.5242143e-03 -5.2256305e-03  5.7156757e-03
 -3.8803243e-03 -2.7025002e-03 -5.3294557e-03  3.6593908e-03
  1.2022030e-03  2.0543062e-03  3.6286730e-03 -4.2388579e-03
  9.3810156e-04  3.1124533e-04 -1.0505852e-03  6.0309721e-03
 -1.8713907e-03 -6.5661373e-04  5.4852008e-03 -5.6367280e-05
 -8.3871577e-03  5.0945715e-03  7.5416686e-03  1.3558607e-04
 -1.6845705e-03  6.6173291e-03 -1.7382887e-03 -2.0188144e-03
 -2.4683857e-03  2.1965760e-03  3.7409100e-03 -6.3673048e-03
 -2.4141187e-03 -2.1519405e-03  1.4604822e-03  2.2677877e-03
  2.5373404e-03 -3.8142968e-03  2.2048308e-03  1.6934010e-03
 -6.6408335e-04 -8.3337435e-03 -2.2385465e-03 -5.1566968e-03
 -3.5533358e-03  2.3890578e-04  3.9576860e-03  3.1923535e-03
 -1.390991

In [ ]:
# sample of output distribution
sampled_indicies = tf.random.categorical(pred, num_samples=1)

# reshape all array and convert all integers to numbers to see actual numbers.
sampled_indicies = np.reshape(sampled_indicies, (1,-1))[0]
predicted_chars = int_to_text(sampled_indicies)

predicted_chars #prediction of model

'ov;((b{11CJ——jRS\ny{xS.–{a3W)G{tmIO|!A:\'13d,jlASN2?4\'ivst—32“pTy!v/rY.J–{NbSELs1r2!I—u”"PIf{{Y’II\'?:2'

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# where checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only = True)

In [ ]:
history = model.fit(data, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
11/11 [==============================] - 2s 82ms/step - loss: 3.7308
Epoch 2/100
11/11 [==============================] - 1s 76ms/step - loss: 3.4156
Epoch 3/100
11/11 [==============================] - 1s 77ms/step - loss: 3.2979
Epoch 4/100
11/11 [==============================] - 1s 97ms/step - loss: 3.0105
Epoch 5/100
11/11 [==============================] - 1s 73ms/step - loss: 2.6903
Epoch 6/100
11/11 [==============================] - 1s 77ms/step - loss: 2.4686
Epoch 7/100
11/11 [==============================] - 1s 74ms/step - loss: 2.3599
Epoch 8/100
11/11 [==============================] - 1s 76ms/step - loss: 2.2508
Epoch 9/100
11/11 [==============================] - 1s 74ms/step - loss: 2.1662
Epoch 10/100
11/11 [==============================] - 1s 76ms/step - loss: 2.0918
Epoch 11/100
11/11 [==============================] - 1s 79ms/step - loss: 2.0347
Epoch 12/100
11/11 [==============================] - 1s 77ms/step - loss: 1.9746
Epoch 13/100
11/11 [=====

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
  # evaluations step(generating text using the learned model)

  # number of characters to generate
  num_generate = 800

  # convert number to string
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # empty string to store results
  text_generated = []

  # low temperature = predictable text
  # higher temperauter = suprising text
  # just experiment
  temperature = 1.0

  # bath size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using categorical distribution to predict the character returned by the model.
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # pass predicted character as the next input to the model along with hidden test
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: shrek
shrek turns and sees the Seven Dwarves with Snow White on the table.
SHREK
Oh, no, no, no. Des! Overy ona.
DONKEY
C curns a sowne be she backs to see that.
SHREK
Oh, thus one of thoseld in.
CUT TO: EXT. MARYING. FOREST. DALT.. FARQUAAD’S TEER.
Fiona s is now walks towards the windmible.
I believe, I believe, people in the back. I was happen a hind off the table.
DWARF
Where are ww me.
Lord Farquaad puckers his lips all, fortally from her.
DONKEY
Hey, this is really oghe the dragon tale creatures! FARQUAAD
Indeed. All right, ogre. I'll make you a dead. Go on it's in there?
MAN 2
All right. Do o just a sperf.
She ase weadon' on his over a hill and you can see Shrek's cottage.
DONKEY
Whoa! Looke ot Dour found a mean, beet, I have a bit of a confession to make. Donkeys don't hav
